In [339]:
from importlib import reload
import functions
reload(functions)
from functions import get_flight_ids, load_json_file,load_parquet_file,process_json_file,  process_parquet_file, remove_constant_columns
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from joblib import dump, load
from concurrent.futures import ProcessPoolExecutor

# 1. Split Train-Test Data

In [333]:
data_dir = 'data' 
flight_ids = get_flight_ids(data_dir)
flight_ids.remove('StateDescriptions')
print(flight_ids)

['08412468-26ac-4777-9afb-4671f426277b', '0b3f3902-2c04-4625-8576-3bb963e3d709', '0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79', '1675a16e-a2a3-4038-9007-50b0b26a685c', '21194a58-8a4d-4d0f-a4a9-e374393183b4', '28bd3cd3-1d6a-403f-ab8a-83efaf260dd0', '2a2467dd-9cb2-45de-a64f-f182395b3d1a', '39b2c145-c49f-470b-8280-d253fa98153f', '663f573a-74c5-4368-b60b-1fb433cd835d', '8ac99efe-b70b-4b4b-983b-6064fe37c67b', '8c36586f-94e9-4ae9-8384-0f3342008677', '92b2d28b-21e4-498c-b6dd-c27a47716a25', '9a2e5b24-1d93-47ef-bd90-0fae0d719df7', 'a366ff0e-ac1e-4632-821f-594ee8750b90', 'a376807a-82d3-4526-b19f-98d4b3f9078b', 'b5a540db-434b-4c3d-86dd-4668d40586c2', 'd76bb0eb-bc08-4b35-8c1f-37369452083d', 'ef4852a4-fcfe-429b-b753-d11e2ad08cac', 'f40f71de-5cc2-4719-8a5a-abcf950cbd71']


In [334]:
test_ids = [
    "0b3f3902-2c04-4625-8576-3bb963e3d709",
    "663f573a-74c5-4368-b60b-1fb433cd835d",
    "8c36586f-94e9-4ae9-8384-0f3342008677",
    "a376807a-82d3-4526-b19f-98d4b3f9078b",
    "d76bb0eb-bc08-4b35-8c1f-37369452083d",
    "f40f71de-5cc2-4719-8a5a-abcf950cbd71"
]

#put the rest into train_ids first
train_ids = []
for i in flight_ids:
    if i not in test_ids:
        train_ids.append(i)

#split train_ids into train and validation ids

train_ids = [
    '08412468-26ac-4777-9afb-4671f426277b', 
    '0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79', 
    '1675a16e-a2a3-4038-9007-50b0b26a685c', 
    '21194a58-8a4d-4d0f-a4a9-e374393183b4', 
    '28bd3cd3-1d6a-403f-ab8a-83efaf260dd0', 
    '2a2467dd-9cb2-45de-a64f-f182395b3d1a', 
    '39b2c145-c49f-470b-8280-d253fa98153f', 
    '8ac99efe-b70b-4b4b-983b-6064fe37c67b', 
    '92b2d28b-21e4-498c-b6dd-c27a47716a25',
    '9a2e5b24-1d93-47ef-bd90-0fae0d719df7'
    ]

validate_ids=[
    'a366ff0e-ac1e-4632-821f-594ee8750b90',
    'b5a540db-434b-4c3d-86dd-4668d40586c2', 
    'ef4852a4-fcfe-429b-b753-d11e2ad08cac'
]

# 2. Train Data Process

## 2.1 Process and save the json files

In [340]:
data_dir = 'data'
output_dir = 'train_data'


if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def save_json_data(flight_id):
    json_path = os.path.join(data_dir, f"{flight_id}.json")

    df = load_json_file(json_path)
    
    df = process_json_file(df)
    
    output_path = os.path.join(output_dir, f"{flight_id}.csv")
    df.to_csv(output_path, index=False)
    print(f"Saved {flight_id} data to {output_path}")

for flight_id in train_ids:
    save_json_data(flight_id)

Saved 08412468-26ac-4777-9afb-4671f426277b data to train_data/08412468-26ac-4777-9afb-4671f426277b.csv
Saved 0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79 data to train_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79.csv
Saved 1675a16e-a2a3-4038-9007-50b0b26a685c data to train_data/1675a16e-a2a3-4038-9007-50b0b26a685c.csv
Saved 21194a58-8a4d-4d0f-a4a9-e374393183b4 data to train_data/21194a58-8a4d-4d0f-a4a9-e374393183b4.csv
Saved 28bd3cd3-1d6a-403f-ab8a-83efaf260dd0 data to train_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0.csv
Saved 2a2467dd-9cb2-45de-a64f-f182395b3d1a data to train_data/2a2467dd-9cb2-45de-a64f-f182395b3d1a.csv
Saved 39b2c145-c49f-470b-8280-d253fa98153f data to train_data/39b2c145-c49f-470b-8280-d253fa98153f.csv
Saved 8ac99efe-b70b-4b4b-983b-6064fe37c67b data to train_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b.csv
Saved 92b2d28b-21e4-498c-b6dd-c27a47716a25 data to train_data/92b2d28b-21e4-498c-b6dd-c27a47716a25.csv
Saved 9a2e5b24-1d93-47ef-bd90-0fae0d719df7 data to train_data/9a2e5b24-1d

In [346]:
# cache the processed json file
cache_dir = './cache/train_data'  
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in train_ids:
    file_path = os.path.join('./train_data', f"{flight_id}.csv")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.csv', '_json.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_csv(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")


Cached data to ./cache/train_data/08412468-26ac-4777-9afb-4671f426277b_json.joblib
Cached data to ./cache/train_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79_json.joblib
Cached data to ./cache/train_data/1675a16e-a2a3-4038-9007-50b0b26a685c_json.joblib
Cached data to ./cache/train_data/21194a58-8a4d-4d0f-a4a9-e374393183b4_json.joblib
Cached data to ./cache/train_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0_json.joblib
Cached data to ./cache/train_data/2a2467dd-9cb2-45de-a64f-f182395b3d1a_json.joblib
Cached data to ./cache/train_data/39b2c145-c49f-470b-8280-d253fa98153f_json.joblib
Cached data to ./cache/train_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b_json.joblib
Cached data to ./cache/train_data/92b2d28b-21e4-498c-b6dd-c27a47716a25_json.joblib
Cached data to ./cache/train_data/9a2e5b24-1d93-47ef-bd90-0fae0d719df7_json.joblib


## 2.2 Analyze StateDescriptions
Before we deal with the parquet files, first we have to analyze the StateDescription.json file.

In [ ]:
import json
import pandas as pd

def analyze_state_descriptions(json_path):
    # Load JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Flatten JSON data and create a DataFrame
    records = []
    for name, entry in data.items():
        record = {
            'Name': name,
            'DataType': entry.get('dataType', 'Unknown'),
            'Unit': entry.get('unit', 'Unknown'),
            'StateID': entry.get('stateId', 'Unknown'),
            'Persistence': entry.get('isPersistent', False),
        }
        
        # Add default values for different models
        for model, default_value in entry.get('DefaultValues', {}).items():
            record[f'Default_{model}'] = default_value

        # Add display units for different models
        for model, display_unit in entry.get('DefaultDisplayUnits', {}).items():
            record[f'Unit_{model}'] = display_unit

        records.append(record)
    
    df = pd.DataFrame(records)
    
    # Display general information about the DataFrame
    print("General Info:")
    print(df.info())
    
    # Display summary statistics for each column
    print("\nSummary Statistics:")
    print(df.describe(include='all'))
    
    # Count unique data types and their frequency
    print("\nData Type Counts:")
    print(df['DataType'].value_counts())
    
    # Count unique units and their frequency
    print("\nUnit Counts:")
    print(df['Unit'].value_counts())
    
    # Check persistence flag distribution
    print("\nPersistence Flag Distribution:")
    print(df['Persistence'].value_counts())
    
    return df


In [ ]:

json_path = './data/StateDescriptions.json'
df_states = analyze_state_descriptions(json_path)

# save to file
output_path = './state_descriptions_analysis.csv'
df_states.to_csv(output_path, index=False)
print(f"\nData saved to {output_path}")

# Display the first few rows for inspection
print("\nFirst few rows of the data:")
print(df_states.head())

General Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         1746 non-null   object
 1   DataType     1746 non-null   object
 2   Unit         381 non-null    object
 3   StateID      1746 non-null   int64 
 4   Persistence  1746 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 56.4+ KB
None

Summary Statistics:
                       Name DataType Unit       StateID Persistence
count                  1746     1746  381  1.746000e+03        1746
unique                 1746        8   22           NaN           2
top     AddOn_CockpitConfig   Double  0-1           NaN       False
freq                      1     1162   99           NaN        1424
mean                    NaN      NaN  NaN  2.118100e+09         NaN
std                     NaN      NaN  NaN  1.232422e+09         NaN
min                     NaN    

## 2.3 Process and save parquet data

### 2.3.1 Cache parquet raw data

In [335]:
cache_dir = './cache/raw'  
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in flight_ids:
    file_path = os.path.join('./data', f"{flight_id}.parquet")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.parquet', '.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_parquet(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")

./cache/raw/08412468-26ac-4777-9afb-4671f426277b.joblib cached already
./cache/raw/0b3f3902-2c04-4625-8576-3bb963e3d709.joblib cached already
./cache/raw/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79.joblib cached already
./cache/raw/1675a16e-a2a3-4038-9007-50b0b26a685c.joblib cached already
./cache/raw/21194a58-8a4d-4d0f-a4a9-e374393183b4.joblib cached already
./cache/raw/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0.joblib cached already
./cache/raw/2a2467dd-9cb2-45de-a64f-f182395b3d1a.joblib cached already
./cache/raw/39b2c145-c49f-470b-8280-d253fa98153f.joblib cached already
./cache/raw/663f573a-74c5-4368-b60b-1fb433cd835d.joblib cached already
./cache/raw/8ac99efe-b70b-4b4b-983b-6064fe37c67b.joblib cached already
./cache/raw/8c36586f-94e9-4ae9-8384-0f3342008677.joblib cached already
./cache/raw/92b2d28b-21e4-498c-b6dd-c27a47716a25.joblib cached already
./cache/raw/9a2e5b24-1d93-47ef-bd90-0fae0d719df7.joblib cached already
./cache/raw/a366ff0e-ac1e-4632-821f-594ee8750b90.joblib cached already
./cach

### 2.3.2 process parquet data and cache

##### 2.3.2.1 See the structures to adjust the function "process_parquet_file"

In [ ]:

# see the original structure os the parquet file
data = pd.read_parquet('./data/2a2467dd-9cb2-45de-a64f-f182395b3d1a.parquet')
print(data[3501046967])

TimeStamp                         FrameCounter
2024-02-14 06:26:55.328843+00:00  0               0.0
2024-02-14 06:26:55.338647+00:00  1               NaN
2024-02-14 06:26:57.816582+00:00  2               NaN
2024-02-14 06:26:57.831172+00:00  3               NaN
2024-02-14 06:26:57.859734+00:00  4               NaN
                                                 ... 
2024-02-14 06:45:40.456558+00:00  289727          NaN
2024-02-14 06:45:40.469469+00:00  289728          NaN
2024-02-14 06:45:40.488957+00:00  289729          NaN
2024-02-14 06:45:40.502970+00:00  289730          NaN
2024-02-14 06:45:40.510829+00:00  289731          NaN
Name: 3501046967, Length: 205635, dtype: float64


In [309]:
# See the structure of the parquet file after process

df = load_parquet_file('./data/08412468-26ac-4777-9afb-4671f426277b.parquet')
df = process_parquet_file(df)
print(df.head())

                                                                                    6417134     \
TimeStamp                        FrameCounter                                                    
2024-02-14 06:46:58.397576+00:00 0             [1.0, 0.17888563049853373, 0.19257086999022482]   
2024-02-14 06:46:58.416348+00:00 1             [1.0, 0.17888563049853373, 0.19257086999022482]   
2024-02-14 06:47:00.886922+00:00 2             [1.0, 0.17888563049853373, 0.19257086999022482]   
2024-02-14 06:47:00.894921+00:00 3             [1.0, 0.17888563049853373, 0.19257086999022482]   
2024-02-14 06:47:00.926924+00:00 4             [1.0, 0.17888563049853373, 0.19257086999022482]   

                                               6741304     28827303    \
TimeStamp                        FrameCounter                           
2024-02-14 06:46:58.397576+00:00 0                  False         0.0   
2024-02-14 06:46:58.416348+00:00 1                  False         0.0   
2024-02-14 06:47:00.8

In [ ]:
# See the structure of the parquet file after remove unnecessay columns

df= remove_constant_columns(df)

MultiIndex([('2024-02-14 06:46:58.397576+00:00',      0),
            ('2024-02-14 06:46:58.416348+00:00',      1),
            ('2024-02-14 06:47:00.886922+00:00',      2),
            ('2024-02-14 06:47:00.894921+00:00',      3),
            ('2024-02-14 06:47:00.926924+00:00',      4),
            ('2024-02-14 06:47:00.946964+00:00',      5),
            ('2024-02-14 06:47:00.950532+00:00',      6),
            ('2024-02-14 06:47:00.961775+00:00',      7),
            ('2024-02-14 06:47:00.978790+00:00',      8),
            ('2024-02-14 06:47:00.984390+00:00',      9),
            ...
            ('2024-02-14 07:03:21.102460+00:00', 253960),
            ('2024-02-14 07:03:21.115700+00:00', 253961),
            ('2024-02-14 07:03:21.135562+00:00', 253962),
            ('2024-02-14 07:03:21.149007+00:00', 253963),
            ('2024-02-14 07:03:21.168594+00:00', 253964),
            ('2024-02-14 07:03:21.182601+00:00', 253965),
            ('2024-02-14 07:03:21.202792+00:00', 253966)

In [314]:
print(df.head()) 

                                               117182271   156875280   \
TimeStamp                        FrameCounter                           
2024-02-14 06:46:58.397576+00:00 0               2.312171       False   
2024-02-14 06:46:58.416348+00:00 1               2.312171       False   
2024-02-14 06:47:00.886922+00:00 2               2.309550       False   
2024-02-14 06:47:00.894921+00:00 3               2.309550       False   
2024-02-14 06:47:00.926924+00:00 4               2.309550       False   

                                                353872548   381802901   \
TimeStamp                        FrameCounter                            
2024-02-14 06:46:58.397576+00:00 0             135500000.0         1.0   
2024-02-14 06:46:58.416348+00:00 1             135500000.0         1.0   
2024-02-14 06:47:00.886922+00:00 2             135500000.0         1.0   
2024-02-14 06:47:00.894921+00:00 3             135500000.0         1.0   
2024-02-14 06:47:00.926924+00:00 4          

##### 2.3.2.2 (Optional) Here there is some mistake that the files are saved to .csv, but remaining the same with .parquet is better. 
Since reprocessing it took a lot of time, so I just wrote the next part to transfer from csv to parquet file.
If you haven't processed the data, skip this part and next (Convert "_parquet.csv" to ".parquet"), just run the part of " Load from cache and process parquet data"

In [ ]:
data_dir = 'data'
output_dir = 'test_data'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to process and save Parquet file data
def save_parquet_data(flight_id):
    parquet_path = os.path.join(data_dir, f"{flight_id}.parquet")

    df = load_parquet_file(parquet_path)
     
    df = process_parquet_file(df)  

    df = remove_constant_columns(df)
    
    # Define the base output file name
    output_path = os.path.join(output_dir, f"{flight_id}_parquet.csv")
    
    # Save the DataFrame as a CSV file
    df.to_csv(output_path, index=False)
    print(f"Saved {flight_id} Parquet data to {output_path}")

# Process each Parquet file and save it
for flight_id in train_ids: 
    # check if the file exists
    if os.path.exists(output_path):
        print(f"File {output_path} already exists. Skipping save.")
    else:
        save_parquet_data(flight_id)
        



File test_data/08412468-26ac-4777-9afb-4671f426277b_parquet.csv already exists. Skipping save.
File test_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79_parquet.csv already exists. Skipping save.
File test_data/1675a16e-a2a3-4038-9007-50b0b26a685c_parquet.csv already exists. Skipping save.
File test_data/21194a58-8a4d-4d0f-a4a9-e374393183b4_parquet.csv already exists. Skipping save.
File test_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0_parquet.csv already exists. Skipping save.
File test_data/2a2467dd-9cb2-45de-a64f-f182395b3d1a_parquet.csv already exists. Skipping save.
File test_data/39b2c145-c49f-470b-8280-d253fa98153f_parquet.csv already exists. Skipping save.
File test_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b_parquet.csv already exists. Skipping save.
Saved 92b2d28b-21e4-498c-b6dd-c27a47716a25 Parquet data to test_data/92b2d28b-21e4-498c-b6dd-c27a47716a25_parquet.csv
Saved 92b2d28b-21e4-498c-b6dd-c27a47716a25 Parquet data to test_data/92b2d28b-21e4-498c-b6dd-c27a47716a25_parquet.csv
Save

##### 2.3.2.3 (Optional) Convert "_parquet.csv" to ".parquet"
If you need to process the parquet data, skip this part. Run next part

In [ ]:

input_dir = "./train_data" 
output_dir = "./train_data" 

os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.endswith("_parquet.csv"): 
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name.replace("_parquet.csv", ".parquet"))
        
        try:
            df = pd.read_csv(input_path)
            df.to_parquet(output_path, index=False)
            print(f"Converted {file_name} to {output_path}")
            
            os.remove(input_path)
            print(f"Deleted original file: {input_path}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

Converted 92b2d28b-21e4-498c-b6dd-c27a47716a25_parquet.csv to ./train_data/92b2d28b-21e4-498c-b6dd-c27a47716a25.parquet
Deleted original file: ./train_data/92b2d28b-21e4-498c-b6dd-c27a47716a25_parquet.csv
Converted 0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79_parquet.csv to ./train_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79.parquet
Deleted original file: ./train_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79_parquet.csv
Converted 8ac99efe-b70b-4b4b-983b-6064fe37c67b_parquet.csv to ./train_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b.parquet
Deleted original file: ./train_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b_parquet.csv
Converted 28bd3cd3-1d6a-403f-ab8a-83efaf260dd0_parquet.csv to ./train_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0.parquet
Deleted original file: ./train_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0_parquet.csv
Converted 2a2467dd-9cb2-45de-a64f-f182395b3d1a_parquet.csv to ./train_data/2a2467dd-9cb2-45de-a64f-f182395b3d1a.parquet
Deleted original file: ./train_data/2a2467dd-9cb2-45de-a64f-

##### 2.3.2.4 Load from cache and process parquet data

In [ ]:
data_dir = './cache/raw'
output_dir = 'test_data'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to process and save Parquet file data
def save_parquet_data(flight_id):
    parquet_path = os.path.join(data_dir, f"{flight_id}.joblib")

    df = load(parquet_path)
     
    df = process_parquet_file(df)  

    df = remove_constant_columns(df)
    
    # Define the base output file name
    output_path = os.path.join(output_dir, f"{flight_id}.parquet")
    
    # Save the DataFrame as a CSV file
    df.to_parquet(output_path, index=False)
    print(f"Saved {flight_id} Parquet data to {output_path}")

# Process each Parquet file and save it
for flight_id in train_ids:
    # check if the file exists
    if os.path.exists(output_path):
        print(f"File {output_path} already exists. Skipping save.")
    else:
        save_parquet_data(flight_id)


In [ ]:
cache_dir = './cache/train'  
os.makedirs(cache_dir, exist_ok=True)  # 如果目录不存在，创建它

# cache parquet_csv file
for flight_id in train_ids:
    file_path = os.path.join('./train_data', f"{flight_id}_parquet.csv")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.csv', '.joblib'))
    
    if os.path.exists(cache_file):
        data = load(cache_file)
        print(f"Loaded cached data from {cache_file}")
    else:
        # 加载原始文件并缓存
        data = pd.read_csv(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")

FileNotFoundError: [Errno 2] No such file or directory: './test_data/08412468-26ac-4777-9afb-4671f426277b_parquet.csv'

In [ ]:
# Test reaading speed

data = load('./cache/9a2e5b24-1d93-47ef-bd90-0fae0d719df7_parquet.joblib')
data.head()

,117182271,156875280,301190512,513685691,518160218,557098463,614270119,637219977,638103778,677695791,...,3966668421,3981973171,4021288050,4054750327,4056189073,4058842283,4061964072,4066202059,4247030081,4294814812
0,5.161555,False,0.0,510.821214,True,[0. 0. 0.71558499],0.006515,511.429802,0.000000,Basic Autorotation,...,[-0.2164971 0.10619898 9.80703578],400.0,4.003269e+06,369.629338,284.829662,[ 0.9999952 0.00244388 0.00148651 -0.00119296],1.0,0.955034,0.002615,0.0
1,5.161555,False,0.0,510.821214,True,[0. 0. 0.71558499],0.006515,511.429802,0.000000,Basic Autorotation,...,[-0.2164971 0.10619898 9.80703578],400.0,4.003269e+06,369.629338,284.829662,[ 0.99999562 0.0024262 0.00136185 -0.0010058 ],1.0,0.953079,0.002615,0.0
2,5.161634,False,0.0,510.910498,True,[0. 0. 0.71558499],0.000000,511.517236,-0.049156,Basic Autorotation,...,[-0.14433302 0.10619898 9.80836326],400.0,4.000000e+06,381.718000,284.829082,[ 9.99996147e-01 2.66921047e-03 6.07066059e-...,1.0,0.950147,0.000000,0.0
3,5.161634,False,0.0,510.910498,True,[0. 0. 0.71558499],0.000000,511.517236,-0.051131,Basic Autorotation,...,[-0.14433263 0.10619898 9.80836326],400.0,4.000000e+06,381.718000,284.829082,[ 9.99996147e-01 2.66921047e-03 6.07066059e-...,1.0,0.950147,0.000000,0.0
4,5.161634,False,0.0,510.910498,True,[0. 0. 0.71558499],0.000000,511.517236,-0.063683,Basic Autorotation,...,[-0.14433263 0.10619898 9.80836326],400.0,4.000000e+06,399.580000,284.829082,[ 9.99996147e-01 2.66921047e-03 6.07066059e-...,1.0,0.950147,0.000000,0.0


### 2.3.3 Cache parquet processed data

In [347]:
cache_dir = './cache/train_data'  
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in train_ids:
    file_path = os.path.join('./train_data', f"{flight_id}.parquet")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.parquet', '_parquet.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_parquet(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")

Cached data to ./cache/train_data/08412468-26ac-4777-9afb-4671f426277b_parquet.joblib
Cached data to ./cache/train_data/0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79_parquet.joblib
Cached data to ./cache/train_data/1675a16e-a2a3-4038-9007-50b0b26a685c_parquet.joblib
Cached data to ./cache/train_data/21194a58-8a4d-4d0f-a4a9-e374393183b4_parquet.joblib
Cached data to ./cache/train_data/28bd3cd3-1d6a-403f-ab8a-83efaf260dd0_parquet.joblib
Cached data to ./cache/train_data/2a2467dd-9cb2-45de-a64f-f182395b3d1a_parquet.joblib
Cached data to ./cache/train_data/39b2c145-c49f-470b-8280-d253fa98153f_parquet.joblib
Cached data to ./cache/train_data/8ac99efe-b70b-4b4b-983b-6064fe37c67b_parquet.joblib
Cached data to ./cache/train_data/92b2d28b-21e4-498c-b6dd-c27a47716a25_parquet.joblib
Cached data to ./cache/train_data/9a2e5b24-1d93-47ef-bd90-0fae0d719df7_parquet.joblib


# 3. Visualize the data